<a href="https://colab.research.google.com/github/DarkBeshka/DarkBeshka.github.io/blob/main/ChatBotAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-telegram-bot --upgrade

In [ ]:
import random
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import logging
from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
# BOT_CONFIG = {
#     'intents': {
#         'hello': {
#             'examples': ['Привет!', 'Здарова', 'Хей-хей!!'],
#             'responses': ['Хай', 'Добрый вечер!', 'Здравствуйте!']
#         },
#         'bye': {
#             'examples': ['Пока', 'Увидимся!', 'Покеда'],
#             'responses': ['До свидания', 'Прощайте', 'Сайонара!']
#         }
#     }
# }
json_file_path = "/content/THISBOT_CONFIG2.json"

with open(json_file_path, 'r') as f:
  BOT_CONFIG = json.load(f)

In [ ]:
with open('/content/BOT_CONFIG1.json', 'w') as f:
   json.dump(BOT_CONFIG, f, ensure_ascii=False, indent=3)

In [ ]:
def clean(text):
  text = text.lower()
  cleaned_text = ''
  for ch in text:
    if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      cleaned_text = cleaned_text + ch
  return cleaned_text

def get_intent(text):
  for intent in BOT_CONFIG['intents'].keys():
    for example in BOT_CONFIG['intents'][intent]['examples']:
      w1 = clean(example)
      w2 = clean(text)
      if nltk.edit_distance(w1, w2) / max(len(w1), len(w2)) < 0.4:
        return intent
  return 'интент не найден'

In [ ]:
X = []            # набор всех текстов, которые у нас есть (для векторайзера(превращает текст в цифры))
y = []            # набор интентов этих текстов. циклом заполняем это:

for intent in BOT_CONFIG['intents'].keys():
    try:
        for example in BOT_CONFIG['intents'][intent]['examples']:
            X.append(example)
            y.append(intent)
    except:
        pass

len(X), len(y), len(set(y))

(1196, 1196, 150)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)  #random_state мы можем зафиксировать для того, чтобы всегда генерировались случайные подвыборки
len(X_train), len(X_test)               # разделяем наш датасет на две части: для обучения и для тренировки

(956, 240)

In [ ]:
#vectorizer=TfidfVectorizer(preprocessor=clean, ngram_range=(1,6), analyzer='char_wb') #+randomforest-> train 0.98, test 0.59, +logreg->train 0.91, test 0.6
vectorizer = CountVectorizer(ngram_range=(1,3), analyzer='char_wb', min_df=0.00001, max_df=0.7)   #используем наш векторайзер, превращающий текст в цифры
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)  #здесь нет fit_transform, чтобы модель не училась на тестовой выборке

len(vectorizer.get_feature_names())  # посмотрим, какой длины получился словарь

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


2729

In [ ]:
#X_train_show=vectorizer.fit_transform(["How are you?", "how are you doing, hey"])
#X_train_show.toarray()
#vectorizer.get_feature_names()

array([[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 1, 0],
       [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 0, 1]])

In [ ]:
# параметры можно было бы перебрать с помощью gridsearch и pipeline
clf = MLPClassifier(max_iter=300).fit(X_train_vect, y_train) 
#log_reg = RandomForestClassifier()   # test 0.67
#log_reg.fit(X_train_vect, y_train)
clf.score(X_train_vect, y_train)         # смотрим результат обучения на тренировочной выборке

0.9916317991631799

In [ ]:
clf.score(X_test_vect, y_test)          #смотрим результат обучения на тестовой выборке
                        # результат может отличаться из раза в раз, потому что имеет место элемент рандома

0.7125

In [ ]:
def get_intent_by_model(text):
    return clf.predict(vectorizer.transform([text]))[0]

In [ ]:
def bot(question):
  intent = get_intent_by_model(question)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
# Подключаем нашего бота к телеграмму. Для этого используем https://github.com/python-telegram-bot/python-telegram-bot
# пример взят из examples -> echobot


# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    out_text = bot(clean(update.message.text))  #принимаем текст
    update.message.reply_text(out_text)   #выводим ответ


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("мой_токен")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
main()

2021-11-22 19:02:41,096 - apscheduler.scheduler - INFO - Scheduler started
2021-11-22 19:03:12,448 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-11-22 19:03:12,451 - apscheduler.scheduler - INFO - Scheduler has been shut down
